In [ ]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
    
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

In [2]:
# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [3]:
%load_ext autoreload
%autoreload 2
import os
import cooler
import cooltools
import numpy as np
import bioframe
from pathlib import Path
import multiprocess as mp
import pandas as pd

from copy import copy

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


In [4]:
# ins_samples = {
#     "Hap1-WT-combined.hg19" : "Hap1-WT-combined.mcool",
# }

ins_samples = {
    #cooler locations, some expected at these locations
    # CTCF degron
    "CkoCT442_NT_pool.hg19" : "/data/alv/CTCF_degron/data/CkoCT442-NT-pool.mcool",
    "CkoCT442_IAA_pool.hg19" : "/data/alv/CTCF_degron/data/CkoCT442-IAA-pool.mcool",

#     #polII degron
#     "PolII-NT.hg19" : "/data/alv/polII_degron/data/20200417_remap_polIIdegron/coolers_library_group/PTB2539-NT.hg19.mapq_30.1000.mcool",
#     "PolII-IAA.hg19" : "/data/alv/polII_degron/data/20200417_remap_polIIdegron/coolers_library_group/PTB2539-IAA.hg19.mapq_30.1000.mcool",

#     #AAVS1 clone
#     "mutControl-NT.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/CkoCT442-AAVS1-NT-pool.mcool",
#     "mutControl-IAA.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/CkoCT442-AAVS1-IAA-pool.mcool",

#     #DDX55 clones
#     "mutDDX55-NT.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/DDX55-clones-NT.hg19.mapq_30.1000.mcool",
#     "mutDDX55-IAA.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/DDX55-clones-IAA.hg19.mapq_30.1000.mcool",

#     #TAF5L clones
#     "mutTAF5L-NT.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/TAF5L-clones-NT.hg19.mapq_30.1000.mcool",
#     "mutTAF5L-IAA.hg19" : "/data/alv/CTCF_degron/data/DDX55-TAF5L-ctrl-pool/TAF5L-clones-IAA.hg19.mapq_30.1000.mcool",

#     #siCONTROL
#     "siControl-NT.hg19" : "/data/alv/CTCF_degron/data/siCTRL-NT.hg19.mapq_30.1000.mcool",
#     "siControl-IAA.hg19" : "/data/alv/CTCF_degron/data/siCTRL-IAA.hg19.mapq_30.1000.mcool",

#     #siDDX55
#     "siDDX55-NT.hg19" : "/data/alv/CTCF_degron/data/siDDX55-NT.hg19.mapq_30.1000.mcool",
#     "siDDX55-IAA.hg19" : "/data/alv/CTCF_degron/data/siDDX55-IAA.hg19.mapq_30.1000.mcool",

#     #siTAF5L
#     "siTAF5L-NT.hg19" : "/data/alv/CTCF_degron/data/siTAF5L-NT.hg19.mapq_30.1000.mcool",
#     "siTAF5L-IAA.hg19" : "/data/alv/CTCF_degron/data/siTAF5L-IAA.hg19.mapq_30.1000.mcool",

#     #RAD21 degron
#     "RAD21-NT.hg19" : "/data/alv/CTCF_degron/data/RAD21-AID-NT.hg19.mapq_30.1000.mcool",
#     "RAD21-IAA.hg19" : "/data/alv/CTCF_degron/data/RAD21-AID-IAA-6H.hg19.mapq_30.1000.mcool",

#     #PlaB splicing inhibition
#     "CtrlPlaB-NT.hg19" : "/data/alv/CTCF_degron/data/NT-hg19-combined-90000000.mcool",
#     "CtrlPlaB-IAA.hg19" : "/data/alv/CTCF_degron/data/IAA-hg19-combined-90000000.mcool",
#     "PlaB-NT.hg19" : "/data/alv/CTCF_degron/data/NT-PlaB-hg19-combined-90000000.mcool",
#     "PlaB-IAA.hg19" : "/data/alv/CTCF_degron/data/IAA-PlaB-hg19-combined-90000000.mcool",

#     #compare with WT
#     "Ctrl500M-noTIR1.hg19" : "/data/alv/CTCF_degron/data/WT-44-442-pool/Hap1-WT-combined-500000000.mcool",
#     "Ctrl500M-wtHAP1.hg19" : "/data/alv/CTCF_degron/data/WT-44-442-pool/CkoC44-NO-TIR1-pool.mcool",
#     "Ctrl500M-CT442-NT.hg19" : "/data/alv/CTCF_degron/data/WT-44-442-pool/CkoCT442-NT-pool-500000000.mcool",
#     "Ctrl500M-CT442-IAA.hg19" : "/data/alv/CTCF_degron/data/WT-44-442-pool/CkoCT442-IAA-pool-500000000.mcool",
}

### Read some list of BEDPE interactions here

In [5]:
bedpe = pd.read_csv("EP_50-500kb_with_detailed_CTCF_status_stricter.bedpe",sep="\t")
# fill in some missing bits (without it groupby does not work well):
bedpe.loc[bedpe["strand1"].isnull(),"strand1"] = "."
bedpe.loc[bedpe["strand2"].isnull(),"strand2"] = "."
bedpe.loc[bedpe["status"].isnull(),"status"] = "undef-ctcf"
bedpe = bedpe.astype({"start1":int,"end2":int})
# print a piece ofit:
display(bedpe.head(4))
#
PE_groups = bedpe.groupby(by=["type1","strand1","type2","strand2","status"])
display(PE_groups.size())

,chrom1,start1,end1,type1,strand1,chrom2,start2,end2,type2,strand2,status
0,chr1,762901,762902,tss,-,chr1,816241,817150,enhancer,.,+
1,chr1,762901,762902,tss,-,chr1,969279,969493,enhancer,.,all
2,chr1,762901,762902,tss,-,chr1,1014914,1015649,enhancer,.,all
3,chr1,762970,762971,tss,+,chr1,816241,817150,enhancer,.,+


type1     strand1  type2     strand2  status    
enhancer  .        tss       +        +             1016
                                      -             1528
                                      all           1389
                                      almost+        235
                                      almost-        417
                                      empty         1880
                                      mix+-         2937
                                      undef-ctcf     202
                             -        +              951
                                      -             1606
                                      all           1191
                                      almost+        194
                                      almost-        356
                                      empty         2165
                                      mix+-         2679
                                      undef-ctcf     312
tss       +        enhancer  .        +

### Let's use coolpup as Python API ...

In [6]:
from matplotlib.colors import LogNorm
# https://stackoverflow.com/questions/48625475/python-shifted-logarithmic-colorbar-white-color-offset-to-center
class MidPointLogNorm(LogNorm):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self, vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        result, is_scalar = self.process_value(value)
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.array(np.interp(np.log(value), x, y), mask=result.mask, copy=False)

In [7]:
# import standard python libraries
import seaborn as sns
# import libraries for biological data analysis
from coolpuppy import coolpup as cp
import cooler
import bioframe
# import cooltools
# import cooltools.expected
# from cooltools.lib import plotting

In [8]:
def get_pup(sites, clr, resolution, _cols=["chrom","start","end"] ):
    """
    generate scaled local pile-up for a collection of sites for a given
    cooler (clr) at a given resolution - keeping the padding size equal
    to the interval itself.
    """
    _sites = sites[_cols].reset_index(drop=True).rename( dict(zip(_cols,["chrom","start","end"])), axis=1 )
    cc = cp.CoordCreator(
        _sites,
        resolution=resolution,
        features_format='bed',
        local=True,
        fraction_flank=1.
    )
    pu = cp.PileUpper(
        clr,
        cc,
        control = True,
        rescale = True,
        rescale_size = 199,
    )
    print(f"working on a group of {len(_sites)} size ...")
    # pileupsWithControl
    # pileupsByStrandWithControl
    pup = pu.pileupsWithControl(nproc=18)
    return pup


In [9]:
key_samples = [
    "CkoCT442_NT_pool.hg19",
    "CkoCT442_IAA_pool.hg19",
]
pup_dict = {}
binsize = 5_000

# group genes by expression and size before extracting corresponding profiles of a list of features :
for sample in key_samples:
    print(f"working on {sample} ...")
    pup_dict[sample] = {}
    cname = ins_samples[sample]
    clr = cooler.Cooler(f"{cname}::/resolutions/{binsize}")
    for grp_name, grp_df in PE_groups:
        type1, strand1, type2, strand2, status = grp_name
        print(f"working on a group {grp_name} ...")
        if status in ["+","-","empty"]:
            # run coolpup ...
            pup_dict[sample][grp_name] = get_pup(grp_df, clr, binsize, _cols=["chrom1","start1","end2"] )

working on CkoCT442_NT_pool.hg19 ...
working on a group ('enhancer', '.', 'tss', '+', '+') ...
working on a group of 1016 size ...


INFO:root:('chr13', 'chr13'): 28
INFO:root:('chr9', 'chr9'): 49
INFO:root:('chr11', 'chr11'): 32
INFO:root:('chr10', 'chr10'): 34
INFO:root:('chr7', 'chr7'): 34
INFO:root:('chr22', 'chr22'): 21
INFO:root:('chr20', 'chr20'): 23
INFO:root:('chr14', 'chr14'): 51
INFO:root:('chr4', 'chr4'): 49
INFO:root:('chr6', 'chr6'): 67
INFO:root:('chr12', 'chr12'): 77
INFO:root:('chr19', 'chr19'): 35
INFO:root:('chr18', 'chr18'): 12
INFO:root:('chr2', 'chr2'): 54
INFO:root:('chr8', 'chr8'): 19
INFO:root:('chr16', 'chr16'): 86
INFO:root:('chr1', 'chr1'): 114
INFO:root:('chr5', 'chr5'): 31
INFO:root:('chr17', 'chr17'): 63
INFO:root:('chr15', 'chr15'): 60
INFO:root:('chr3', 'chr3'): 63
INFO:root:Total number of piled up windows: 1002


working on a group ('enhancer', '.', 'tss', '+', '-') ...
working on a group of 1528 size ...


INFO:root:('chr16', 'chr16'): 52
INFO:root:('chr8', 'chr8'): 40
INFO:root:('chr9', 'chr9'): 65
INFO:root:('chr10', 'chr10'): 59
INFO:root:('chr4', 'chr4'): 57
INFO:root:('chr5', 'chr5'): 60
INFO:root:('chr21', 'chr21'): 13
INFO:root:('chr19', 'chr19'): 67
INFO:root:('chr3', 'chr3'): 74
INFO:root:('chr20', 'chr20'): 33
INFO:root:('chr22', 'chr22'): 46
INFO:root:('chr18', 'chr18'): 26
INFO:root:('chr2', 'chr2'): 116
INFO:root:('chr12', 'chr12'): 78
INFO:root:('chr1', 'chr1'): 155
INFO:root:('chr7', 'chr7'): 91
INFO:root:('chr15', 'chr15'): 64
INFO:root:('chr13', 'chr13'): 61
INFO:root:('chr14', 'chr14'): 51
INFO:root:('chr6', 'chr6'): 119
INFO:root:('chr11', 'chr11'): 84
INFO:root:('chr17', 'chr17'): 110
INFO:root:Total number of piled up windows: 1521


working on a group ('enhancer', '.', 'tss', '+', 'all') ...
working on a group ('enhancer', '.', 'tss', '+', 'almost+') ...
working on a group ('enhancer', '.', 'tss', '+', 'almost-') ...
working on a group ('enhancer', '.', 'tss', '+', 'empty') ...
working on a group of 1880 size ...


...

working on a group ('tss', '-', 'enhancer', '.', '-') ...
working on a group of 1049 size ...


INFO:root:('chr8', 'chr8'): 34
INFO:root:('chr7', 'chr7'): 52
INFO:root:('chr14', 'chr14'): 74
INFO:root:('chr5', 'chr5'): 46
INFO:root:('chr19', 'chr19'): 31
INFO:root:('chr6', 'chr6'): 70
INFO:root:('chr22', 'chr22'): 17
INFO:root:('chr20', 'chr20'): 20
INFO:root:('chr21', 'chr21'): 30
INFO:root:('chr2', 'chr2'): 37
INFO:root:('chr3', 'chr3'): 79
INFO:root:('chr4', 'chr4'): 71
INFO:root:('chr1', 'chr1'): 118
INFO:root:('chr17', 'chr17'): 77
INFO:root:('chr9', 'chr9'): 56
INFO:root:('chr10', 'chr10'): 9
INFO:root:('chr11', 'chr11'): 40
INFO:root:('chr12', 'chr12'): 65
INFO:root:('chr18', 'chr18'): 8
INFO:root:('chr16', 'chr16'): 30
INFO:root:('chr13', 'chr13'): 46
INFO:root:('chr15', 'chr15'): 35
INFO:root:Total number of piled up windows: 1045


working on a group ('tss', '-', 'enhancer', '.', 'all') ...
working on a group ('tss', '-', 'enhancer', '.', 'almost+') ...
working on a group ('tss', '-', 'enhancer', '.', 'almost-') ...
working on a group ('tss', '-', 'enhancer', '.', 'empty') ...
working on a group of 2031 size ...


INFO:root:('chr18', 'chr18'): 32
INFO:root:('chr16', 'chr16'): 53
INFO:root:('chr17', 'chr17'): 69
INFO:root:('chr19', 'chr19'): 30
INFO:root:('chr21', 'chr21'): 24
INFO:root:('chr15', 'chr15'): 76
INFO:root:('chr14', 'chr14'): 115
INFO:root:('chr22', 'chr22'): 24
INFO:root:('chr13', 'chr13'): 120
INFO:root:('chr8', 'chr8'): 98
INFO:root:('chr20', 'chr20'): 66
INFO:root:('chr11', 'chr11'): 132
INFO:root:('chr10', 'chr10'): 68
INFO:root:('chr7', 'chr7'): 102
INFO:root:('chr4', 'chr4'): 111
INFO:root:('chr3', 'chr3'): 134
INFO:root:('chr12', 'chr12'): 112
INFO:root:('chr2', 'chr2'): 166
INFO:root:('chr6', 'chr6'): 100
INFO:root:('chr9', 'chr9'): 126
INFO:root:('chr1', 'chr1'): 177
INFO:root:('chr5', 'chr5'): 90
INFO:root:Total number of piled up windows: 2025


working on a group ('tss', '-', 'enhancer', '.', 'mix+-') ...
working on a group ('tss', '-', 'enhancer', '.', 'undef-ctcf') ...


In [10]:
import pickle
file_to_store = open("PUP_DICT_5kb_SHIFT_EXP_PromEnhanc_detailed_stricter.pickle", "wb")
pickle.dump(pup_dict, file_to_store)
# Save object to file
file_to_store.close()

### plotting different pileups for different groups ...

In [ ]:
from matplotlib import colors

In [ ]:
def reverse_strand(strand):
    """
    reverse strand: +->-, -->+, .->.
    """
    if strand == "+":
        return "-"
    elif strand == "-":
        return "+"
    else:
        return strand

def flip_int(int_description):
    """
    flip promoter-enhancer pairs into Enhancer to Promoter
    """
    type1, strand1, type2, strand2, *_bs = int_description
    return type2, reverse_strand(strand2), type1, reverse_strand(strand1), *_bs

def flip_PE(pe_tuple):
    """
    flip promoter-enhancer pairs into Enhancer to Promoter
    """
    t1, *_ = pe_tuple
    if t1 != "enhancer":
        return flip_int(pe_tuple)
    else:
        return pe_tuple

In [ ]:
pup_dict["CkoCT442_NT_pool.hg19"]

## Remember to flip CTCF as well ...

In [ ]:
combined_dict = {}
mean_dict = {}
for k,mat in pup_dict["CkoCT442_NT_pool.hg19"].items():
    mat = mat["data"]["all"]
    invk = flip_PE(k)
    invmat = mat[::-1,::-1] if k != invk else mat
    # populate combined_dict now
    if invk not in combined_dict:
        combined_dict[invk] = [ invmat ]
    else:
        combined_dict[invk].append(invmat)
        mean_dict[invk] = np.nanmean(combined_dict[invk],axis=0)


fig = plt.figure(figsize=(12,8),constrained_layout=True)
gs = fig.add_gridspec(2,3)
for idx, (name, mat) in enumerate(mean_dict.items()):
    j, i = idx//2, idx%2
    ax = fig.add_subplot(gs[i,j])
    ax.set_title(name)
    img = ax.imshow(
        mat,
        # norm = MidPointLogNorm(vmin=0.75,vmax=1.25,midpoint=1),
        norm = colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.25),
        cmap="coolwarm",
        interpolation="nearest",            
    )
# plt.colorbar(img)

In [ ]:
combined_dict = {}
mean_dict = {}
for k,mat in pup_dict["CkoCT442_IAA_pool.hg19"].items():
    mat = mat["data"]["all"]
    invk = flip_PE(k)
    invmat = mat[::-1,::-1] if k != invk else mat
    # populate combined_dict now
    if invk not in combined_dict:
        combined_dict[invk] = [ invmat ]
    else:
        combined_dict[invk].append(invmat)
        mean_dict[invk] = np.nanmean(combined_dict[invk],axis=0)


fig = plt.figure(figsize=(12,8),constrained_layout=True)
gs = fig.add_gridspec(2,3)
for idx, (name, mat) in enumerate(mean_dict.items()):
    j, i = idx//2, idx%2
    ax = fig.add_subplot(gs[i,j])
    ax.set_title(name)
    img = ax.imshow(
        mat,
        # norm = MidPointLogNorm(vmin=0.75,vmax=1.25,midpoint=1),
        norm = colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.25),
        cmap="coolwarm",
        interpolation="nearest",            
    )
# plt.colorbar(img)

In [ ]:
combined_dict = {}
mean_dict = {}
for k,mat in pup_dict["Ctrl500M-CT442-IAA.hg19"]["data"].to_dict().items():
    invk = flip_PE(k)
    invmat = mat[::-1,::-1] if k != invk else mat
    # populate combined_dict now
    if invk not in combined_dict:
        combined_dict[invk] = [ invmat ]
    else:
        combined_dict[invk].append(invmat)
        mean_dict[invk] = np.nanmean(combined_dict[invk],axis=0)


fig = plt.figure(figsize=(7,8),constrained_layout=True)
gs = fig.add_gridspec(2,2)
for idx, (name, mat) in enumerate(mean_dict.items()):
    j, i = idx//2, idx%2
    ax = fig.add_subplot(gs[i,j])
    ax.set_title(name)
    img = ax.imshow(
        mat,
        # norm = MidPointLogNorm(vmin=0.75,vmax=1.25,midpoint=1),
        norm = colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.25),
        cmap="coolwarm",
        interpolation="nearest",            
    )
# plt.colorbar(img)

In [ ]:
fig = plt.figure(figsize=(12,8),constrained_layout=True)
gs = fig.add_gridspec(2,len(PE_groups)//2)

_yyyy = pup_dict["RAD21-IAA.hg19"]#.reset_index()#.set_index(["gexp","gsize","orientation"])

for idx, (name, _) in enumerate(PE_groups):
    j, i = idx//2, idx%2
    ax = fig.add_subplot(gs[i,j])
    ax.set_title(name)
    pup_heatmap = _yyyy.loc[name,"data"].values[0]
    img = ax.imshow(
        pup_heatmap,
        # norm = MidPointLogNorm(vmin=0.75,vmax=1.25,midpoint=1),
        norm = colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.25),
        cmap="coolwarm",
        interpolation="nearest",            
    )
# plt.colorbar(img)

In [ ]:
fig = plt.figure(figsize=(12,8),constrained_layout=True)
gs = fig.add_gridspec(2,len(PE_groups)//2)

_yyyy = pup_dict["Ctrl500M-CT442-IAA.hg19"]#.reset_index()#.set_index(["gexp","gsize","orientation"])

for idx, (name, _) in enumerate(PE_groups):
    j, i = idx//2, idx%2
    ax = fig.add_subplot(gs[i,j])
    ax.set_title(name)
    pup_heatmap = _yyyy.loc[name,"data"].values[0]
    img = ax.imshow(
        pup_heatmap,
        # norm = MidPointLogNorm(vmin=0.75,vmax=1.25,midpoint=1),
        norm = colors.TwoSlopeNorm(vmin=0.75, vcenter=1, vmax=1.25),
        cmap="coolwarm",
        interpolation="nearest",            
    )
# plt.colorbar(img)